# Current version: 0.2.1 (unfinished)

From 2024, updates to the dataset are handled and stored in a separate file. This is that file (previously, all Art500k dataset processing was done in *art500k.csv*, the file now renamed to *art500k_initial*).

To-do steps:

- Group artists together with their aliases (e.g. "Rembrandt" and "Rembrandt van Rijn" are two different instances in the dataset)
- Filter out artists that are not painters (e.g. photographers, sculptors, engravers, etc.)

In [1]:
import numpy as np
import pandas as pd

url_v_latest = "https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/artists.csv"
url_v_latest_art500k_artists = "https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/art500k_artists.csv"
artists = pd.read_csv(url_v_latest)
art500k_artists = pd.read_csv(url_v_latest_art500k_artists, dtype={'Type': str})

In [2]:
import helper_functions  #From helper_functions.py

## 2024.01.07 - : Use measures to find artists with multiple names (aliases)

If we take a look at popular artists in the dataset, for example Rembrandt:

In [54]:
import numpy as np
import pandas as pd

url_v_01_11 = "https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/saves/art500k_artists_0_3.csv"
art500k_artists = pd.read_csv(url_v_01_11, dtype={'Type': str})

In [55]:
art500k_artists[art500k_artists['artist'].str.contains("Rembrandt")]['artist'].unique()

array(['Rembrandt Peale', 'Rembrandt', 'after Rembrandt van Rijn',
       'Rembrandt Harmensz. van Rijn', 'Rembrandt van Rijn',
       'British 19th Century after Rembrandt van Rijn',
       'Richard Houston after Rembrandt van Rijn',
       'William Byron after Rembrandt van Rijn',
       'Georg Friedrich Schmidt after Rembrandt van Rijn',
       'Jonas Suyderhoff after Rembrandt van Rijn',
       'Timothy Cole after Rembrandt van Rijn',
       'Richard Earlom after Rembrandt van Rijn',
       'Govaert FLINCK (Discípulo de Rembrandt)'], dtype=object)

There are multiple entries for Rembrandt: *Rembrandt*, *Rembrandt van Rijn*,  *Rembrandt Harmensz. van Rijn*, *Rembrandt (Rembrandt van Rijn)*, *Rembrandt Harmensz van Rijn (Dutch)*, *Rembrandt (Rembrandt van Rijn)|Rembrandt (Rembrandt van Rijn)*. We need to combine entries for one artists if there are more than 1.<br>
However, generally this is not trivial to find. 

The other problem is processing other instances such as "X after Y". I believe for these cases, LLMs may be the most useful.

As of now, we tackle this problem by using a combination of measures to find artist aliases.

Considered measures:

* Fuzzy string matching (Levenshtein distance) between artist names. 
* Basic string containment (other artists names containing one word artist names, e.g. Rembrandt).
* Token-Based Matching (TBM) (Jaccard similarity) between artist names.
* Named Entity Recognition (NER) (Spacy) to find artist names from text, then apply Coreference Resolution to link pronouns and other expressions to the correct entities.
* LLMs to find artist names from text.


Other considerations: <br>
* Phonetic matching: This could be helpful when an artist's name is spelled differently in different languages, e.g. "Č" (Czech) / "Ch" (English) / "cs" (Hungarian). Even if this is the case for some instances, we should find these with the Levenshtein distance search. <br>
* Online available resources for aliases, web scraping, etc.
* Custom rules (e.g "... and his workshop", "... and his circle", etc.)

NER:

In [10]:
import spacy

#Example
data = {
    'author_name': ['Rembrandt', 'Rembrandt van Rijn', 'Rembrandt Peale', 'Michelangelo', 'Michelangelo Buonarroti', 'Michelangelo Merisi da Caravaggio', 'Caravaggio', 'Caravaggio, Michelangelo Merisi da', 'Caravaggio, Michelangelo Merisi da (Italian, Milan or Caravaggio 1571-1610 Porto Ercole)', 'Leonardo', 'Leonardo da Vinci'],
}
df = pd.DataFrame(data)

nlp = spacy.load("en_core_web_sm") #English only
aliases = {}

for name in df['author_name']:
    doc = nlp(name)
    for ent in doc.ents:
        if ent.label_ == 'PERSON':
            aliases.setdefault(name, set()).add(ent.text)
            aliases.setdefault(ent.text, set()).add(name)

aliases = {key: list(value) for key, value in aliases.items()}
aliases

{'Rembrandt van Rijn': ['Rembrandt van Rijn'],
 'Rembrandt Peale': ['Rembrandt Peale'],
 'Michelangelo': ['Michelangelo'],
 'Michelangelo Buonarroti': ['Michelangelo Buonarroti'],
 'Michelangelo Merisi da Caravaggio': ['Michelangelo Merisi da'],
 'Michelangelo Merisi da': ['Michelangelo Merisi da Caravaggio',
  'Caravaggio, Michelangelo Merisi da (Italian, Milan or Caravaggio 1571-1610 Porto Ercole)'],
 'Caravaggio, Michelangelo Merisi da': ['Michelangelo Merisi', 'Caravaggio'],
 'Caravaggio': ['Caravaggio, Michelangelo Merisi da',
  'Caravaggio, Michelangelo Merisi da (Italian, Milan or Caravaggio 1571-1610 Porto Ercole)'],
 'Michelangelo Merisi': ['Caravaggio, Michelangelo Merisi da'],
 'Caravaggio, Michelangelo Merisi da (Italian, Milan or Caravaggio 1571-1610 Porto Ercole)': ['Michelangelo Merisi da',
  'Caravaggio'],
 'Leonardo': ['Leonardo'],
 'Leonardo da Vinci': ['Leonardo da Vinci']}

This seems to leave out many 1-word-alias cases, but it is a start.

In [22]:
import spacy

nlp = spacy.load("en_core_web_sm") #English only
aliases = {}

for name in art500k_artists[art500k_artists['artist'].str.contains("Rembrandt")]['artist'].unique():
    doc = nlp(name)
    for ent in doc.ents:
        if ent.label_ == 'PERSON':
            aliases.setdefault(name, set()).add(ent.text)
            aliases.setdefault(ent.text, set()).add(name)

aliases = {key: list(value) for key, value in aliases.items()}
aliases

{'Rembrandt Peale': ['Rembrandt Peale'],
 'after Rembrandt van Rijn': ['Rembrandt van Rijn'],
 'Rembrandt van Rijn': ['William Luson Thomas|Sir John Gilbert|Rembrandt (Rembrandt van Rijn)',
  'Jonas Suyderhoff after Rembrandt van Rijn',
  'Timothy Cole after Rembrandt van Rijn',
  'after Rembrandt van Rijn',
  'Richard Houston after Rembrandt van Rijn',
  'Charles Turner after Rembrandt van Rijn',
  'Georg Friedrich Schmidt after Rembrandt van Rijn',
  'Costantino Cumano after Rembrandt van Rijn',
  'Richard Earlom after Rembrandt van Rijn',
  'Captain William E. Baillie|Rembrandt (Rembrandt van Rijn)',
  'Jean Pierre de Frey|Rembrandt (Rembrandt van Rijn)',
  'Jan Georg (Joris) van Vliet|Rembrandt (Rembrandt van Rijn)',
  'Rembrandt van Rijn',
  'Rembrandt (Rembrandt van Rijn)|Rembrandt (Rembrandt van Rijn)',
  'Rembrandt (Rembrandt van Rijn)|Andrea Mantegna|Rembrandt (Rembrandt van Rijn)',
  'William Byron after Rembrandt van Rijn',
  'British 19th Century after Rembrandt van Rijn'],

The "after", "attributed to", "|", "follower of" cause big problems. We need to find a way to deal with these first.

In [40]:
multiple_names = []
for name in art500k_artists['artist'].unique():
    if "|" in name:
        multiple_names.append(name)

multiple_names

['Albrecht Dürer|Albrecht Dürer',
 'Nina de Garis Davies|Norman de Garis Davies',
 'Vienna|Du Paquier period',
 'Du Paquier period|Vienna',
 'Naotane Taikei|Yukinaka|Honjo Yoshitane',
 'John Milton|David Bogue|Myles Birket Foster|John Milton',
 'Henry N. Hooper and Company|O.P. Drake',
 'Nakagawa Isshō|Sadakuni',
 'Yasumitsu|Iwamoto Konkan',
 'Sukemitsu of Bizen|Yasumitsu|Iwamoto Konkan',
 'Fusamune of Soshu|Sōheishi Sōtensai',
 'Thomas Tompion|Nathanial Delander',
 'Henry Atkin|James Purdey the Elder|Abbe Robins|Peter Gumbrell',
 'Toshimasa|Kajikawa|Kuniyoshi',
 'Jean Henri Riesener|Jean-Gotfritt Mercklein',
 'Hanabusa Itchō|Kajikawa',
 "Johann Schott|Hans Wechtlin|Ulrich Pinder|Unidentified Weaver's Mark|Unidentified Weaver's Mark",
 'Antoine Watteau|François Boucher',
 'Brewster & Co.|Herman Stahmer',
 'Johann Georg Christoph Fries|Angelo Quaglio',
 'Ruth Whittier Shute|Samuel Addison Shute',
 'Brewster & Co.|Paul Rodissart',
 'Brewster & Co.|Channing Britton',
 'Brice Thomas|Franço

In [15]:
from transformers import pipeline

# Initialize the pipeline
generator = pipeline('text-generation', model='gpt2')
string = "This painting was painted after Leonardo by Rafael. The painting was painted by"
# Generate text
text = generator(string, max_length=len(string)+1)[0]['generated_text']

print(text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


This painting was painted after Leonardo by Rafael. The painting was painted by an Italian painter, who came from Milan, and his painting was the result of a long association with the Raffaele and with the Artemesons. A group of the artists worked together and made the painting. From the start of the 12th century, the artist wrote, he was not afraid of the artists of


In [35]:
from transformers import GPT2Tokenizer, pipeline

# Initialize the tokenizer and the pipeline
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
generator = pipeline('text-generation', model='gpt2')

pairs = []
for name in art500k_artists[art500k_artists['artist'].str.contains("Rembrandt")]['artist'].unique():
    string = "The description of this painting's author is: " + name + ". The yes-or-no answer to the question 'Is the painter of this painting known or unknown?' is:"
    tokens = tokenizer.encode(string, return_tensors='pt')
    text = generator(string, max_length=tokens.shape[1]+1)[0]['generated_text'];
    answer = text.split(string)[1]
    pairs.append([name, answer])

pairs

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

[['Rembrandt Peale', ' Yes'],
 ['Rembrandt', ' No'],
 ['after Rembrandt van Rijn', ' of'],
 ['Rembrandt Harmensz. van Rijn', ' Not'],
 ['Rembrandt van Rijn', ' Yes'],
 ['British 19th Century after Rembrandt van Rijn', ' No'],
 ['Richard Houston after Rembrandt van Rijn', " '"],
 ['William Byron after Rembrandt van Rijn', " '"],
 ['Georg Friedrich Schmidt after Rembrandt van Rijn', " '"],
 ['Jonas Suyderhoff after Rembrandt van Rijn', ' Is'],
 ['Timothy Cole after Rembrandt van Rijn', ' The'],
 ['Richard Earlom after Rembrandt van Rijn', ' The'],
 ['School of Rembrandt van Rijn', ' He'],
 ['Rembrandt (Rembrandt van Rijn)', ' a'],
 ['Nicolaes Maes|School of Rembrandt van Rijn', '\n'],
 ['Rembrandt (Rembrandt van Rijn)|Ferdinand Bol', ' An'],
 ['Rembrandt (Rembrandt van Rijn)|Nicolaes Maes', ' Yes'],
 ['Rembrandt (Rembrandt van Rijn)|Andrea Mantegna|Rembrandt (Rembrandt van Rijn)',
  " '"],
 ['Attributed to Rembrandt Peale', ' "'],
 ['Costantino Cumano after Rembrandt van Rijn', ' he'],
 

In [7]:
import difflib

# Function to calculate similarity between two strings
def similarity(s1, s2):
    return difflib.SequenceMatcher(None, s1, s2).ratio()


cases = art500k_artists[art500k_artists['artist'].str.contains("Rembrandt")]['artist'].unique()
# Cluster cases based on similarity
clusters = {}
for case in cases:
    assigned_cluster = False
    for cluster_center, original_painter in clusters.items():
        if similarity(case, cluster_center) > 0.6:
            clusters[cluster_center].append(case)
            assigned_cluster = True
            break
    if not assigned_cluster:
        clusters[case] = [case]

# Print the clusters
for cluster_center, cluster_cases in clusters.items():
    print(f"Cluster center: {cluster_center}")
    print(f"Cases in cluster: {', '.join(cluster_cases)}")
    print()


Cluster center: Rembrandt Peale
Cases in cluster: Rembrandt Peale, Rembrandt, Rembrandt van Rijn, Attributed to Rembrandt Peale

Cluster center: after Rembrandt van Rijn
Cases in cluster: after Rembrandt van Rijn, Rembrandt Harmensz. van Rijn, British 19th Century after Rembrandt van Rijn, Richard Houston after Rembrandt van Rijn, William Byron after Rembrandt van Rijn, Georg Friedrich Schmidt after Rembrandt van Rijn, Jonas Suyderhoff after Rembrandt van Rijn, Timothy Cole after Rembrandt van Rijn, Richard Earlom after Rembrandt van Rijn, School of Rembrandt van Rijn, Rembrandt (Rembrandt van Rijn), Nicolaes Maes|School of Rembrandt van Rijn, Rembrandt (Rembrandt van Rijn)|Ferdinand Bol, Rembrandt (Rembrandt van Rijn)|Nicolaes Maes, Costantino Cumano after Rembrandt van Rijn, follower of Rembrandt Harmensz. van Rijn, Charles Turner after Rembrandt van Rijn, Rembrandt Harmensz van Rijn (Dutch, After the School of Rembrandt Harmenszoon van Rijn, school of Rembrandt Harmensz. van Rijn, F

This is quite inconvinient and not always accurate (see: ['Follower of Rembrandt van Rijn', ' Yes'])

In [6]:
pd.DataFrame(art500k_artists['artist']).to_csv('art500k_artists.txt', sep=";" , index=False)
#Rembrandt
pd.DataFrame(art500k_artists[art500k_artists['artist'].str.contains("Rembrandt")]['artist'].unique()).to_csv('rembrandt.txt', sep=";" , index=False)

### Current step: Using ChatGPT to find artist aliases and filter unknowns

In [22]:
artists_for_GPT = art500k_artists[1507:]['artist'].reset_index(drop=True)
for i in range(0, len(artists_for_GPT), 150):
    string = "datasets/generated/artists_for_GPT_" + str(i/150) + ".txt"
    pd.DataFrame(artists_for_GPT[i:i+150]).to_csv(string, sep=";" , index=False)
    #Remove first line
    # ...
    with open(string, 'r', encoding='utf-8') as fin:
        data = fin.readlines()

    with open(string, 'w', encoding='utf-8') as fout:
        fout.writelines(data[1:])

In [40]:
art500k_artists.loc[0:1730, "Type"]

0       None
1       None
2       None
3       None
4       None
        ... 
1726    None
1727    None
1728    None
1729    None
1730    None
Name: Type, Length: 1731, dtype: object

In [12]:
art500k_artists[art500k_artists['artist'].str.contains("Juan Carre")]['artist']

209      Juan Carreño de Miranda
38745    Juan Carreno De Miranda
Name: artist, dtype: object

## 2024.01.24: Look through "|" cases, make minor fixes

The amount of cases where there is a "|" in the artist name (suggesting multiple artists as possible painters) used to be quite high, but from the previous update where 18000 instances of basically no information were removed, there is only one such case.

In [56]:
url_v_01_11 = "https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/saves/art500k_artists_0_3.csv"
art500k_artists = pd.read_csv(url_v_01_11, dtype={'Type': str})

cases = (art500k_artists[art500k_artists['artist'].str.contains("|", regex=False)]['artist']).unique()
for case in cases:
    print(case)

Albrecht Dürer|Albrecht Dürer


In [57]:
art500k_artists_copy = helper_functions.art500k_combine_instances(art500k_artists, "Albrecht Dürer","Albrecht Dürer|Albrecht Dürer")
art500k_artists_copy = helper_functions.art500k_combine_instances(art500k_artists_copy, "Albrecht Dürer","Albrecht D_rer")
art500k_artists_copy = helper_functions.art500k_combine_instances(art500k_artists_copy, "Albrecht Dürer","Albrecht Dürer (German")
art500k_artists_copy = helper_functions.art500k_combine_instances(art500k_artists_copy, "Albrecht Durer","Albrecht Dürer") #For WikiArt, this name is better

In [58]:
art500k_artists_copy.to_csv('datasets/saves/art500k_artists_0_3.csv', index=False)
art500k_artists_copy.to_csv('datasets/art500k_artists.csv', index=False)

Manually added Type for Dürer.

## 2024.01.24 Filter out totally empty rows (aside from name, nationality, contemporary y/n and type) except if the artist is in WikiArt

In [45]:
wikiart_artists = pd.read_csv("datasets/wikiart_artists.csv")

drops = art500k_artists.copy()
drops = drops[(drops.drop(columns=['artist', 'Nationality'])).notna().any(axis=1)==False]
drops2 = drops.copy() #Cannot change dataframe while iterating over it
for artist in drops['artist']:
    if artist in wikiart_artists['artist'].unique():
        drops2 = drops2[drops2['artist'] != artist]

In [48]:
wikiart_artists = pd.read_csv("datasets/wikiart_artists.csv")

drops = art500k_artists.copy()
drops = drops[(drops.drop(columns=['artist', 'Nationality', 'Contemporary', 'Type'])).notna().any(axis=1)==False]
drops2 = drops.copy() #Cannot change dataframe while iterating over it
for artist in drops['artist']:
    if artist in wikiart_artists['artist'].unique():
        drops2 = drops2[drops2['artist'] != artist]

art500k_artists_copy = art500k_artists.copy()
art500k_artists_copy = art500k_artists_copy[~(art500k_artists_copy['artist'].isin(drops2['artist']))].reset_index(drop=True)
art500k_artists_copy

,artist,Nationality,PaintingSchool,ArtMovement,Influencedby,Influencedon,Pupils,Teachers,FriendsandCoworkers,FirstYear,LastYear,Places,PlacesYears,StylesYears,StylesCount,PlacesCount,Contemporary,Type
0,Gustave Courbet,French,NaN,{Realism:272},"Rembrandt,Caravaggio,Diego Velazquez,Peter Pau...","Edouard Manet,Claude Monet,Pierre-Auguste Reno...",NaN,NaN,NaN,1830.0,1877.0,"London, Montpellier, Moscow, CA, UK, Norway, D...","France:1841-1876,Switzerland:1844-1874,Lille:1...","Realism:1835-1877,Romanticism:1830-1849","{Realism:257}, {Romanticism:13}","{France:88},{Switzerland:7},{Lille:8},{Paris:4...",No,Painting/Sculpture
1,Auguste Rodin,French,NaN,"{Modern art:3},{Impressionism:91}","Michelangelo,Donatello,","Georgia O'Keeffe,Man Ray,Aristide Maillol,Olex...","Constantin Brancusi,",NaN,NaN,1865.0,1985.0,"London, CA, UK, Switzerland, Lisbon, US, Germa...","France:1865-1889,Paris:1865-1898,CA:1891-1891,...",Impressionism:1865-1905,{Impressionism:90},"{France:52},{Paris:15},{Brussels:2},{Belgium:1...",NaN,Painting/Sculpture
2,Frida Kahlo,Mexican,NaN,"{Naïve Art (Primitivism),Surrealism:99}","Amedeo Modigliani,Diego Rivera,Jose Clemente O...","Judy Chicago,Georgia O'Keeffe,Feminist Art,",NaN,NaN,NaN,1922.0,1954.0,"CA, LA, New York, US, New Orleans, Washington ...","Mexico:1927-1954,San Francisco:1931-1933,Mexic...","Naïve Art (Primitivism):1922-1954,Surrealism:1...","{Naïve Art (Primitivism):99}, {Surrealism:15}","{Mexico:50},{San Francisco:6},{New York:4},{Me...",No,Painting/Sculpture
3,Banksy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011.0,2011.0,"Los Angeles, London, UK, Palestine, California...","London:2011-2011,UK:2011-2011",NaN,NaN,"{Palestine:1},{Los Angeles:3},{California:3},{...",Yes,Painting/Sculpture
4,El Greco,"Spanish,Greek",Cretan School,"{Spanish Renaissance:1},{Renaissance:2},{Manne...","Byzantine Art,","Expressionism,Cubism,Eugene Delacroix,Edouard ...",NaN,"Titian,","Giulio Clovio,",1568.0,1614.0,"Seville, London, Illescas, Romania, Moscow, Gr...","Spain:1577-1599,London:1600-1600,UK:1600-1600,...",Mannerism (Late Renaissance):1568-1600,"{Renaissance:2}, {XVI CenturySpanish Painting:...","{Spain:75},{Boston:1},{MA:1},{US:27},{Museo de...",No,Painting/Sculpture
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21024,Théo van Rysselberghe,Belgian,Les XX,{Post-Impressionism:186},NaN,NaN,NaN,NaN,NaN,1880.0,1926.0,"Belgium, Brussels, Netherlands, Otterlo, Hague...","Otterlo:1890-1890,Netherlands:1890-1920,Amster...","Post-Impressionism:1900-1926,Impressionism:188...","{Post-Impressionism:65}, {Impressionism:34}, {...","{Otterlo:2},{Netherlands:6},{Amsterdam:1},{Utr...",NaN,NaN
21025,János Mattis-Teutsch,"Hungarian,Romanian",NaN,"{Art Nouveau:1},{Socialist realism:1},{Abstrac...",NaN,NaN,NaN,NaN,NaN,1909.0,1947.0,NaN,NaN,"Constructivism:1925-1930,Abstract Art:1918-192...","{Constructivism:11}, {Abstract Art:61}, {Expre...",NaN,NaN,NaN
21026,Édouard Debat-Ponsan,French,NaN,"{Academic art:1},{Academic Art:11}",NaN,NaN,NaN,NaN,NaN,1876.0,1902.0,NaN,NaN,Academicism:1876-1902,{Academicism:11},NaN,No,NaN
21027,Juan de Valdés Leal,Spanish,NaN,{Baroque:17},"Virgin-Mary,Christianity,Christianity,saints-a...","Museo del Prado, Madrid, Spain,Museum of Fine ...",NaN,NaN,Artists2/Juan De Valdes Leal/The Imaculate Con...,1650.0,1700.0,"Seville, US, St. Louis, Russia, Saint Petersbu...",NaN,NaN,{Baroque:17},"{St. Louis:1},{MO:1},{US:1},{Seville:4},{Spain...",No,NaN


In [50]:
art500k_artists_copy.to_csv('datasets/art500k_artists.csv', index=False)
art500k_artists_copy.to_csv('datasets/saves/art500k_artists_0_3.csv', index=False)

# End of version 0_2

## 2024.01.20-23 Remove "Main" from locations (e.g. London, Main -> Main was detected as a separate location)

In [3]:
import numpy as np
import pandas as pd

url_v_01_10 = "https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/saves/art500k_artists_0_2.csv"
art500k_artists = pd.read_csv(url_v_01_10, dtype={'Type': str})

import helper_functions  #From helper_functions.py

The idea: if in certain columns (e.g. "Places") there is a certain value contained (e.g. "Main") but not an exception (e.g. "Maine"), then a switch function is called


In [4]:
#NOTE: this is now placed in the helper_functions.py file
from helper_functions import row_contains_values_switch

#Switch function: #NOTE: A more general, word switch function is now placed in the helper_functions.py file
def switch_function_exclude_main(row_as_series, column_name):
    import re
    row = row_as_series.copy()

    if not isinstance(row_as_series[column_name], str): #For example, if it is NaN (float)
        return row

    if column_name == "Places": 
        row[column_name] = row_as_series[column_name].replace(", Main", "").replace(" Main,","")#Deal with all cases (beginning, end, middle)
        if row[column_name] == "Main":#One case can happen: if there is only one place, "Main"
            row[column_name] = ""
    if column_name == "PlacesYears":
        expressions = re.findall(r"Main:\d+-\d+|$", row_as_series[column_name])
        expression = expressions[0] if len(expressions) > 0 else ""
        if expression != "":
            row[column_name] = row_as_series[column_name].replace(","+expression, "").replace(expression+",","")
            if row[column_name] == expression: #If only one place, "Main"
                row[column_name] = ""
    if column_name == "PlacesCount":
        expressions = re.findall(r"\{Main:\d+\}", row_as_series[column_name])
        expression = expressions[0] if len(expressions) > 0 else ""
        if expression != "":
            row[column_name] = row_as_series[column_name].replace(","+expression, "").replace(expression+",","")
            if row[column_name] == expression:
                row[column_name] = ""
    return row

<details><summary><u>Testing:</u></summary>

```python

def test_row_switching():
    df = pd.DataFrame({
        'Places': ['umm, Maine, USA', 'London, Main', 'One, Main, Two', 'Saint-Germaine', 'USA', 'Mains, Main', 'umm, Main, USA'],
        'PlacesYears': ['umm:...,Main:1990-2000,', 'a:_,Main:2-12,b', 'Main:1990-2000', 'Main:1990-2000', 'Main:1990-2000,Maine:1990-2000', 'Mains:1990-2000,Main:1990-2000', 'umm:...,Main:1990-2000,Maine:1990-2000'],
        'PlacesCount': ['{Main:12},{yes:2}', '', '{Other:},{Main:2}', '{Mains:12}', '{Maine:12}', '{Mains:12},{Main:12}', '{Main:12},{yes:2},{Maine:12}']
    })

    df_result = df.apply(lambda row: row_contains_values_switch(row, ['Places', 'PlacesYears', 'PlacesCount'], ['Main'], exceptions=['Maine', 'Germain'], switch_function=switch_function_exclude_main), axis=1)

    assert df_result['Places'][0] == 'umm, Maine, USA' #1) Don't change anything, because Maine is an exception
    assert df_result['Places'][1] == 'London' #2) Remove Main
    assert df_result['Places'][2] == 'One, Two'#2)
    assert df_result['Places'][3] == 'Saint-Germaine' #1)
    assert df_result['Places'][4] == 'USA' #3) Don't change anything, because there is no Main
    assert df_result['Places'][5] == 'Mains' #2)
    assert df_result['Places'][6] == 'umm, Main, USA' #1b) Don't change anything, because Maine is an exception in another column

    assert df_result['PlacesYears'][0] == 'umm:...,Main:1990-2000,'
    assert df_result['PlacesYears'][1] == 'a:_,b'
    assert df_result['PlacesYears'][2] == ''
    assert df_result['PlacesYears'][3] == 'Main:1990-2000'
    assert df_result['PlacesYears'][4] == 'Main:1990-2000,Maine:1990-2000'
    assert df_result['PlacesYears'][5] == 'Mains:1990-2000'
    assert df_result['PlacesYears'][6] == 'umm:...,Main:1990-2000,Maine:1990-2000'

    assert df_result['PlacesCount'][0] == '{Main:12},{yes:2}' #1) Don't change anything, because Maine is an exception in another column
    assert df_result['PlacesCount'][1] == ''
    assert df_result['PlacesCount'][2] == '{Other:}'
    assert df_result['PlacesCount'][3] == '{Mains:12}'
    assert df_result['PlacesCount'][4] == '{Maine:12}'
    assert df_result['PlacesCount'][5] == '{Mains:12}'
    assert df_result['PlacesCount'][6] == '{Main:12},{yes:2},{Maine:12}'
    
    return df_result

test_row_switching()

```

</details>

Now let's use it on the dataset:

In [7]:
art500k_artists_copy = art500k_artists.apply(lambda row: row_contains_values_switch(row,columns = ["Places", "PlacesYears", "PlacesCount"], texts=["Main"], exceptions=["Maine", "am Main","Germain"], switch_function=switch_function_exclude_main), axis=1)

In [10]:
art500k_artists_copy.to_csv("datasets/saves/art500k_artists_0_2.csv", index=False)
art500k_artists_copy.to_csv("datasets/art500k_artists.csv", index=False)

## Update 2024.01.18: Add few more cases to help combine with WikiArt

(see *datasets_notebook.ipynb* 2024.01.16- update)

In [6]:
import numpy as np
import pandas as pd

url_v_01_10 = "https://raw.githubusercontent.com/me9hanics/PainterPalette/main/datasets/saves/art500k_artists_0_2.csv"
art500k_artists = pd.read_csv(url_v_01_10, dtype={'Type': str})

In [7]:
pairs = {
    "Juan Carreño de Miranda": "Juan Carreno De Miranda",
    'Albert Ràfols-Casamada': 'Albert Rafols Casamada',
    'Francisco De Zurbaran': 'Francisco de Zurbarán',
    'Andrés de Santa Maria': 'Andres De Santa Maria', 
    'Jean-Honoré Fragonard':'Jean Honore Fragonard',
    'Théo van Rysselberghe': 'Theo Van Rysselberghe',
    'János Mattis-Teutsch': 'Janos Mattis Teutsch',
    'Édouard Debat-Ponsan': 'Edouard Debat Ponsan',
    'Juan de Valdés Leal': 'Juan De Valdes Leal',
    'Park Seo Bo': 'Park Seo-bo'
}
for key, value in pairs.items():
    art500k_artists = helper_functions.art500k_combine_instances(art500k_artists, key, value)
art500k_artists[-10:]

,artist,Nationality,PaintingSchool,ArtMovement,Influencedby,Influencedon,Pupils,Teachers,FriendsandCoworkers,FirstYear,LastYear,Places,PlacesYears,StylesYears,StylesCount,PlacesCount,Contemporary,Type
39983,Juan Carreño de Miranda,Spanish,NaN,{Baroque:26},NaN,NaN,NaN,NaN,NaN,1656.0,1684.0,"Valenciennes, Museo del Prado, Austria, Budape...","France:1666-1666,Museo del Prado:1680-1680,Mad...",Baroque:1656-1684,{Baroque:26},"{Valenciennes:1},{France:2},{Museo del Prado:1...",No,Painting/Sculpture
39984,Albert Ràfols-Casamada,Spanish,NaN,{Art Informel:28},NaN,NaN,NaN,NaN,NaN,1858.0,2004.0,NaN,NaN,Art Informel:1858-2004,{Art Informel:28},NaN,NaN,NaN
39985,Francisco De Zurbaran,Spanish,NaN,{Baroque:96},"Caravaggio,","Gustave Courbet,",NaN,"Francisco Pacheco,",NaN,1625.0,1664.0,"Hungary, Museo del Prado, Paris, Barcelona, B...","Grenoble:1626-1640,France:1626-1661,Seville:16...",Baroque:1625-1664,{Baroque:94},"{Grenoble:7},{France:19},{Seville:31},{Spain:3...",No,NaN
39986,Andrés de Santa Maria,Colombian,NaN,{Impressionism:10},"Jean-Francois Millet,Gustave Courbet,",NaN,NaN,NaN,NaN,1894.0,1942.0,NaN,NaN,Impressionism:1894-1942,{Impressionism:10},NaN,NaN,NaN
39987,Jean-Honoré Fragonard,French,NaN,"{Rococo:72},{Renaissance:1}",NaN,NaN,NaN,NaN,NaN,1750.0,1790.0,"Netherlands, Paris,London, Pasadena, Moscow, ...","France:1753-1782,Paris:1765-1778,Russia:1760-1...",Rococo:1750-1790,{Rococo:70},"{France:21},{Paris:8},{Moscow:1},{Russia:3},{S...",No,Painting/Sculpture
39988,Théo van Rysselberghe,Belgian,Les XX,{Post-Impressionism:186},NaN,NaN,NaN,NaN,NaN,1880.0,1926.0,"Belgium, Brussels, Netherlands, Otterlo, Hague...","Otterlo:1890-1890,Netherlands:1890-1920,Amster...","Post-Impressionism:1900-1926,Impressionism:188...","{Post-Impressionism:65}, {Impressionism:34}, {...","{Otterlo:2},{Netherlands:6},{Amsterdam:1},{Utr...",NaN,NaN
39989,János Mattis-Teutsch,"Hungarian,Romanian",NaN,"{Art Nouveau:1},{Socialist realism:1},{Abstrac...",NaN,NaN,NaN,NaN,NaN,1909.0,1947.0,NaN,NaN,"Constructivism:1925-1930,Abstract Art:1918-192...","{Constructivism:11}, {Abstract Art:61}, {Expre...",NaN,NaN,NaN
39990,Édouard Debat-Ponsan,French,NaN,"{Academic art:1},{Academic Art:11}",NaN,NaN,NaN,NaN,NaN,1876.0,1902.0,NaN,NaN,Academicism:1876-1902,{Academicism:11},NaN,No,NaN
39991,Juan de Valdés Leal,Spanish,NaN,{Baroque:17},"Virgin-Mary,Christianity,Christianity,saints-a...","Museo del Prado, Madrid, Spain,Museum of Fine ...",NaN,NaN,Artists2/Juan De Valdes Leal/The Imaculate Con...,1650.0,1700.0,"Seville, US, St. Louis, Russia, Saint Petersbu...",NaN,NaN,{Baroque:17},"{St. Louis:1},{MO:1},{US:1},{Seville:4},{Spain...",No,NaN
39992,Park Seo Bo,South Korean,NaN,"{Korean Informel ,Dansaekhwa (Korean Monochrom...",NaN,NaN,NaN,NaN,NaN,1968.0,2007.0,Korea,NaN,Minimalism:1968-2007,{Minimalism:18},{Korea:1},Yes,NaN


In [9]:
art500k_artists.to_csv('datasets/art500k_artists.csv', index=False)
art500k_artists.to_csv('datasets/saves/art500k_artists_0_2.csv', index=False)

## Update 2024.01.13-15
Add contemporary, and profession columns, and start removing unknown painters such as masters. 

In [41]:
art500k_artists['Contemporary'] = None
art500k_artists['Type'] = None
for index, artist in art500k_artists.iterrows():
    if pd.notnull(artist["LastYear"]):
        if artist["LastYear"] >= 2000:
            art500k_artists.loc[index, 'Contemporary'] = "Yes"
        elif artist["LastYear"] < 1980:
            art500k_artists.loc[index, 'Contemporary'] = "No"

art500k_artists.loc[0:1730, "Type"] = "Painting/Sculpture"
art500k_artists.loc[1731:2760, "Type"] = "Graffiti"; art500k_artists.loc[1731:2760, "Contemporary"] = "Yes"
art500k_artists.loc[2761:2900, "Type"] = "Design/Photography/Miscellaneous"
art500k_artists.loc[2900:2902, "Type"] = "Sculpture"; art500k_artists.loc[2900:2902, "Contemporary"] = "No"
art500k_artists.loc[2903:2907, "Type"] = "Painting"; art500k_artists.loc[2903:2907, "Contemporary"] = "No"
art500k_artists.loc[2908:2908, "Type"] = "Photography"; art500k_artists.loc[2908:2908, "Contemporary"] = "No"
art500k_artists.loc[2909:2911, "Type"] = "Design"; art500k_artists.loc[2909:2911, "Contemporary"] = "Yes"
art500k_artists.loc[2912:2912, "Type"] = "Painting"; art500k_artists.loc[2912:2912, "Contemporary"] = "No"
art500k_artists.loc[2913:2913, "Type"] = "Engraver/Miscellaneous"; art500k_artists.loc[2913:2913, "Contemporary"] = "No"
art500k_artists.loc[2914:2914, "Type"] = "Graphic Design"
art500k_artists.loc[2915:2915, "Type"] = "Graffiti"; art500k_artists.loc[2915:2915, "Contemporary"] = "Yes"
art500k_artists.loc[2916:2916, "Type"] = "Miscellaneous"; art500k_artists.loc[2916:2916, "Contemporary"] = "Yes"
art500k_artists.loc[2917:2917, "Type"] = "Painting"; art500k_artists.loc[2917:2917, "Contemporary"] = "Yes"

In [45]:
art500k_artists[art500k_artists['artist'].str.contains("Master")]['artist']

732                                         Bedford Master
757                                    Master of Frankfurt
956                                           Master E. S.
1020                    Master of the Virgo inter Virgines
1059                                     Master of Alkmaar
                               ...                        
36895                              Second Master of Bierge
37375                                     Master of Pedret
37655                                      Budapest Master
38569    Italian 16th Century or Master of the Victoria...
39106                                       Master Francke
Name: artist, Length: 290, dtype: object

Remove masters

In [63]:
masters = art500k_artists[art500k_artists['artist'].str.contains("Master") | art500k_artists['artist'].str.contains("master")]['artist']
masters_list=((masters).to_list()); masters_list.remove('Master Francke')
art500k_artists = art500k_artists[~(art500k_artists['artist'].isin(masters_list))].reset_index(drop=True)


In [64]:
art500k_artists.to_csv('datasets/art500k_artists.csv', index=False)
art500k_artists.to_csv('datasets/saves/art500k_artists_0_2.csv', index=False)

# End of 0_1 version of the Art500k artists dataset

## Update 2024.01.12-13

Minor artist change, to test the instance combination method + remove quotation marks (") from artist names.

In [28]:
art500k_artists[art500k_artists['artist'].str.contains("Gustavo Dall")]

,artist,Nationality,PaintingSchool,ArtMovement,Influencedby,Influencedon,Pupils,Teachers,FriendsandCoworkers,FirstYear,LastYear,Places,PlacesYears,StylesYears,StylesCount,PlacesCount
1314,Gustavo Dall'Ara,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1875.0,1923.0,NaN,NaN,NaN,NaN,NaN
36434,Gustavo Dall'ara,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1910.0,1913.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
art500k_modified = helper_functions.art500k_combine_instances(df=art500k_artists, primary_artist_name="Gustavo Dall'Ara", secondary_artist_name="Gustavo Dall'ara")
art500k_modified[art500k_modified['artist'].str.contains("Gustavo Dall")]

In [30]:
art500k_artists = art500k_modified
art500k_modified['artist'].to_csv('art500k_artists.txt', sep=";" , index=False)
art500k_modified.to_csv('datasets/saves/art500k_artists_0_1.csv', index=False)

In [32]:
art500k_artists[art500k_artists['artist'].str.contains('"')]['artist']


1836                                "CHRIS ""DAZE"" ELLIS"
8069                         "Alejandro ""Mono"" González"
8222                                        "Nemi ""UHU"""
8902                            """Rafael Lozano-Hemmer"""
12257          "Giovanni Battista Trotti (""Il Malosso"")"
12567    "Richard Cosway|Mary ""Perdita"" Robinson|Will...
12626         "Giovanni Battista Discepoli (""Il Zoppo"")"
12722           "Giovanni Battista Crespi (""Il Cerano"")"
12742    "Bernardino Rodriguez (""Bernardino Siciliano"")"
13053    "Francesco Monti (""Il Brescianino delle Batta...
13069                             "John ""Warwick"" Smith"
13449         "Giorgio di Giovanni (""Giorgio da Siena"")"
13855              "Giovanni Battista (""Titta"") Lusieri"
13881                   "Giovanni Balducci (""Il Cosci"")"
16443        "Hanna Lachert; ""Ład"" Artists’ Cooperative"
20616                                             """TC"""
21332         "Michelangelo Cerruti (""Il Candelottaro""

In [33]:
art500k_artists['artist'] = art500k_artists['artist'].str.replace('"', '')
art500k_artists['artist'].str.contains('"').sum()

0

In [34]:
art500k_artists['artist'].to_csv('art500k_artists.txt', sep=";" , index=False)
art500k_artists.to_csv('datasets/saves/art500k_artists_0_1.csv', index=False)

In [3]:
art500k_artists[art500k_artists['artist'].str.contains("Marc Bohan")]

,artist,Nationality,PaintingSchool,ArtMovement,Influencedby,Influencedon,Pupils,Teachers,FriendsandCoworkers,FirstYear,LastYear,Places,PlacesYears,StylesYears,StylesCount,PlacesCount
1507,Marc Bohan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1969.0,1969.0,"Paris, France",NaN,NaN,NaN,"{Paris:2},{France:3}"
24700,Marc Bohan for Christian Dior SE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Paris, France",NaN,NaN,NaN,"{Paris:4},{France:4}"


In [6]:
art500k_modified = helper_functions.art500k_combine_instances(df=art500k_artists, primary_artist_name="Marc Bohan", secondary_artist_name="Marc Bohan for Christian Dior SE")
art500k_modified[art500k_modified['artist'].str.contains("Marc Bohan")]

,artist,Nationality,PaintingSchool,ArtMovement,Influencedby,Influencedon,Pupils,Teachers,FriendsandCoworkers,FirstYear,LastYear,Places,PlacesYears,StylesYears,StylesCount,PlacesCount
40295,Marc Bohan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1969.0,1969.0,"Paris, France",NaN,NaN,NaN,"{Paris:6},{France:7}"


In [8]:
art500k_modified.to_csv('datasets/saves/art500k_artists_0_1.csv', index=False)
art500k_modified['artist'].to_csv('art500k_artists.txt', sep=";" , index=False)
art500k_modified.to_csv('datasets/art500k_artists.csv', index=False)

## Update 2024.01.11

One minor change: remove the double "," in StylesYears and PlacesYears.

In [ ]:
#Remove double commas
art500k_artists_copy = art500k_artists.copy()
for index, row in art500k_artists.iterrows():
    dict_like_columns = ['ArtMovement', 'StylesCount','PlacesCount']
    years_columns = ['FirstYear','LastYear','PlacesYears','StylesYears']

    for column in dict_like_columns+years_columns:
        column_value = row[column]
        if type(column_value) == float: #NaN
            continue
        values = [x for x in column_value.split(',') if x != '']
        values_one_comma_string = ",".join(values)
        art500k_artists_copy.at[index, column] = values_one_comma_string

In [184]:
art500k_artists_copy.to_csv("datasets/saves/art500k_artists_0_1.csv", index=False)